In [1]:
import os
import pandas as pd

def leitura_df(path):
    arquivos_csv = [arquivo for arquivo in os.listdir(path) if arquivo.endswith(".csv")]

    dataframes = []  # Lista para armazenar os dataframes de cada arquivo CSV

    for arquivo_csv in arquivos_csv:
        caminho_arquivo = os.path.join(path, arquivo_csv)
        dataframe = pd.read_csv(caminho_arquivo)
        dataframes.append(dataframe)

    df = pd.concat(dataframes, ignore_index=True)
    return df


def ler_arquivo_censo(caminho_arquivo):
    with open(caminho_arquivo, 'r') as arquivo:
        linhas = arquivo.readlines()

    dados = []

    for linha in linhas:
        linha = linha.strip()
        if linha.startswith(('GOVERNO DO', 'SECRETARIA DE', 'SUBSECRETARIA DE', 'DIRETORIA DE', 'GERÊNCIA DE', 'QUADRO', 'REDE PÚBLICA', 'CENSO ESCOLAR', 'RA', 'Tur Matr', 'TOTAL', 'FONTE', 'NOTA', 'Z:')):
            continue
        elif linha == '':
            continue
        else:
            dados.append(linha.split())

    colunas = dados.pop(0)
    df = pd.DataFrame(dados, columns=colunas)

    return df

In [2]:
import pandas as pd

ras_obitos_nascidos = pd.DataFrame({
    'Cidade': [
        'Plano Piloto', 'Gama', 'Taguatinga', 'Brazlândia', 'Sobradinho', 'Planaltina',
        'Paranoá', 'Núcleo Bandeirante', 'Ceilândia', 'Guará', 'Cruzeiro',
        'Samambaia', 'Santa Maria', 'São Sebastião', 'Recanto das Emas', 'Lago Sul',
        'Riacho Fundo II', 'Lago Norte', 'Candangolândia', 'Águas Claras', 'Riacho Fundo',
        'Sudoeste/Octogonal', 'Varjão', 'Park Way', 'SCIA/Estrutural', 'Sobradinho II',
        'Jardim Botânico', 'Itapoã', 'SIA', 'Vicente Pires', 'Fercal', 'Sol Nascente/Pôr do Sol'
    ],
    'RA': [
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
        27, 28, 29, 30, 31, 32
    ]
})


## Obtidos DF

In [3]:
obitos = leitura_df("obtidos/2022")
obitos = obitos.merge(ras_obitos_nascidos, left_on='i_desc_radf_res',right_on='Cidade', how='left')


In [4]:
obitos_fetais = obitos[obitos['i_desc_tipo_obito'] == 'fetal']


In [5]:
#i_qtd_obitos

## Nascidos Vivos

In [6]:
#i_qtd_nasc

In [7]:
nascidos_vivos = leitura_df("nascidos_vivos/2022")
nascidos_vivos = nascidos_vivos.merge(ras_obitos_nascidos, left_on='i_desc_radf_res_mae',right_on='Cidade', how='left')

In [8]:
df_obitos_nasc = obitos.merge(nascidos_vivos, on='RA', how='left')

In [9]:
df_fetal = df_obitos_nasc[df_obitos_nasc['i_desc_tipo_obito'] == 'fetal']
df_fetal_ra = df_fetal.groupby('RA').agg({'i_qtd_obitos': 'sum', 'i_qtd_nasc': 'sum'})
df_fetal_ra['S3'] = (df_fetal_ra['i_qtd_obitos'] / df_fetal_ra['i_qtd_nasc'] * 1000).round(3)

#### Mortes evitaveis

In [10]:
lista_codigo_mortes_evitaveis = pd.read_csv("obtidos/lista_mortes_evitaveis/codigo.csv", sep=",")
obitos['mortes_evitaveis'] = obitos['i_cid_obito'].isin(lista_codigo_mortes_evitaveis['i_cid_obito'])
obitos['mortes_evitaveis'] = obitos['mortes_evitaveis'].fillna(False)


In [11]:
df_obitos_evitaveis = obitos[obitos['mortes_evitaveis'] == True]
df_obitos_evitaveis_ra = df_obitos_evitaveis.groupby('RA').agg(S4=('i_qtd_obitos','sum'))

## Censo Escolar DF

### media turmas ensino fundamental 2019

In [12]:
import pandas as pd

def ler_arquivo(nome_arquivo):
    df = pd.read_csv(nome_arquivo, delimiter=';')
    return df

file_path = 'educacao/media_fundamental_2019.csv'

In [13]:
media_alunos_turma_fundamental = ler_arquivo(file_path)

In [14]:
taxa_defasagem_fundamental = pd.read_csv('educacao/porcentagem_defasagem_idade_serie_fundamentao_2019.csv', delimiter=';')

In [15]:
taxa_reprovacao_fundamental = pd.read_csv('educacao/reprovacao_fundamentao_2019.csv', delimiter=';') 

## Pdad 2021

In [16]:
pdad_2021_mor = pd.read_csv('pdad/2021/PDAD_2021-Moradores.csv', sep=';',decimal=',', encoding='latin-1', low_memory=False)
#pdad_2021_mor['PESO_MOR'] = pdad_2021_mor['PESO_MOR'].str.replace(',', '.').astype(float)
pdad_2021_mor['renda_ind_r'] = pdad_2021_mor['renda_ind_r'].str.replace(',', '.').str.replace(r'[^0-9.]', '0').astype(float)

/var/folders/q5/sd340ftx67vgbyk_3nznc82h0000gn/T/ipykernel_62357/147846973.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  pdad_2021_mor['renda_ind_r'] = pdad_2021_mor['renda_ind_r'].str.replace(',', '.').str.replace(r'[^0-9.]', '0').astype(float)


In [17]:
proj_pop_ra = pdad_2021_mor.groupby('A01ra').sum()['PESO_MOR']

/var/folders/q5/sd340ftx67vgbyk_3nznc82h0000gn/T/ipykernel_62357/1380784816.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  proj_pop_ra = pdad_2021_mor.groupby('A01ra').sum()['PESO_MOR']


In [18]:
#var_se1 = pdad_2021_mor.query('H11 >= 9 and H13 == 1').groupby('A01ra').count()['A01nficha']
var_se1 = pdad_2021_mor.query('H11 >= 9').groupby('A01ra').agg(SE1=('PESO_MOR', 'sum'))

In [19]:
df_var_se1 = pd.merge(var_se1, proj_pop_ra, on='A01ra')
df_var_se1['SE1_porcent'] = df_var_se1['SE1'] / df_var_se1['PESO_MOR'] * 100
df_var_se1.head(3)

,SE1,PESO_MOR,SE1_porcent
A01ra,,,
1,218961.371113,224848.0,97.381952
2,114481.403170,137331.0,83.361661
3,184864.558136,210498.0,87.822477


In [20]:
var_se2 = pdad_2021_mor.query('I22_1 > 0 and I22_1 < 77777 and renda_ind_r < 200').groupby('A01ra').agg(SE2=('PESO_MOR', 'sum'))
var_se3 = pdad_2021_mor.query('I22_1 > 0 and I22_1 < 77777 and renda_ind_r < 100').groupby('A01ra').agg(SE3=('PESO_MOR', 'sum'))
df_var_se2_se3 = pd.merge(var_se2, proj_pop_ra, on='A01ra')
df_var_se2_se3 = pd.merge(df_var_se2_se3, var_se3, on='A01ra')
df_var_se2_se3['SE2_10000'] = df_var_se2_se3['SE2'] / df_var_se2_se3['PESO_MOR'] * 10000
df_var_se2_se3['SE3_10000'] = df_var_se2_se3['SE3'] / df_var_se2_se3['PESO_MOR'] * 10000


In [21]:
var_se4 = pdad_2021_mor.query('idade > 65').groupby('A01ra').agg(SE4=('PESO_MOR', 'sum'))
var_se4 = pd.merge(var_se4, proj_pop_ra, on='A01ra')
var_se4['SE4_percent'] = var_se4['SE4'] / var_se4['PESO_MOR'] * 100
var_se4.head(3)

,SE4,PESO_MOR,SE4_percent
A01ra,,,
1,27901.073650,224848.0,12.408860
2,12072.526316,137331.0,8.790824
3,23566.307351,210498.0,11.195502


In [22]:
var_se5_dependente = pdad_2021_mor.query('idade < 15 or idade > 60').groupby('A01ra').agg(SE5_DEP=('PESO_MOR', 'sum'))
var_se5_ativos = pdad_2021_mor.query('idade > 15 and idade < 60').groupby('A01ra').agg(SE5_ATIV=('PESO_MOR', 'sum'))
var_se5 = pd.merge(var_se5_dependente, var_se5_ativos, on='A01ra')
var_se5['SE5_percent'] = var_se5['SE5_DEP'] / var_se5['SE5_ATIV'] * 100

### Segurança publica

 data = {
    'RA': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
    ],
    'Cidade': [
        'Brasília', 'Gama', 'Taguatinga', 'Brazlândia', 'Sobradinho', 'Planaltina',
        'Paranoá', 'Núcleo Bandeirante', 'Ceilândia', 'Guará', 'Cruzeiro', 'Samambaia',
        'Santa Maria', 'São Sebastião', 'Recanto das Emas', 'Lago Sul', 'Riacho Fundo',
        'Lago Norte', 'Candangolândia', 'Águas Claras', 'Riacho Fundo 2', 'Sudoeste',
        'Varjão do Torto', 'Park Way', 'Estrutural', 'Sobradinho 2', 'Jardim Botânico',
        'Itapoã', 'SIA', 'Vicente Pires', 'Fercal', 'Sol Nascente/Pôr do Sol',
        'Arniqueira'
    ],
    'Ocorrencias': [
        4869, 1145, 3127, 272, 379, 1000, 913, 296, 4701, 825, 94, 2753, 1236, 591,
        1166, 69, 230, 119, 70, 582, 295, 147, 30, 33, 455, 234, 23, 666, 215, 292, 22,
        509, 206
    ]
}

In [23]:
ccp_2022 = pd.read_csv('seguranca/2022/ccp.csv', sep=';', low_memory=False).set_index('RA',inplace=False)
ccp_2022 = ccp_2022.merge(proj_pop_ra, how='left', left_index=True, right_index=True)
ccp_2022['SEG1_1000'] = ccp_2022['Ocorrencias'] / ccp_2022['PESO_MOR'] * 1000

import pandas as pd
data = {
    'RA': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33],
    'Cidade': [
        'Brasília', 'Gama', 'Taguatinga', 'Brazlândia', 'Sobradinho', 'Planaltina',
        'Paranoá', 'Núcleo Bandeirante', 'Ceilândia', 'Guará', 'Cruzeiro', 'Samambaia',
        'Santa Maria', 'São Sebastião', 'Recanto das Emas', 'Lago Sul', 'Riacho Fundo',
        'Lago Norte', 'Candangolândia', 'Águas Claras', 'Riacho Fundo 2', 'Sudoeste',
        'Varjão do Torto', 'Park Way', 'Estrutural', 'Sobradinho 2', 'Jardim Botânico',
        'Itapoã', 'SIA', 'Vicente Pires', 'Fercal', 'Sol Nascente/Pôr do Sol',
        'Arniqueira'
    ],
    'Ocorrencias': [
        8, 6, 17, 8, 3, 22, 9, 4, 37, 4, 1, 19, 17, 24, 23, '-', 4, 2, 3, '-', 5, 2, 2, 1, 12, 9, '-', 10, 1, 5, 4, 17, 6
    ]
}


In [24]:
cvli_2022 = pd.read_csv('seguranca/2022/cvli.csv', sep=';', low_memory=False).set_index('RA',inplace=False)
cvli_2022 = cvli_2022.merge(proj_pop_ra, how='left', left_index=True, right_index=True)
cvli_2022['Ocorrencias'] = cvli_2022['Ocorrencias'].str.replace('-', '0').astype(float)
cvli_2022['SEG2_100000'] = cvli_2022['Ocorrencias'] / cvli_2022['PESO_MOR'] * 100000

# Join final

In [53]:
S3 = df_fetal_ra['S3']
df_obitos_evitaveis_ra_pop = df_obitos_evitaveis_ra.merge(proj_pop_ra, how='left', left_index=True, right_index=True)
df_obitos_evitaveis_ra_pop['S4_10000'] = df_obitos_evitaveis_ra_pop['S4'] / df_obitos_evitaveis_ra_pop['PESO_MOR'] * 10000
S4= df_obitos_evitaveis_ra_pop['S4_10000']
media_alunos_turma_fundamental['E2'] =  media_alunos_turma_fundamental['total matriculas'] / media_alunos_turma_fundamental['turmas']
E2 = media_alunos_turma_fundamental['E2']
taxa_defasagem_fundamental = pd.read_csv('educacao/porcentagem_defasagem_idade_serie_fundamentao_2019.csv', delimiter=';').rename(columns={'% Defasagem Idade-Série Fundamental':'E5'}, inplace=False)
taxa_reprovacao_fundamental = pd.read_csv('educacao/reprovacao_fundamentao_2019.csv', delimiter=';').rename(columns={'Reprovados em %':'E6'}, inplace=False) 
E5 = taxa_defasagem_fundamental['E5']
E6 = taxa_reprovacao_fundamental['E6']
SE1 = df_var_se1['SE1_porcent']
SE2 = df_var_se2_se3['SE2_10000']
SE3 = df_var_se2_se3['SE3_10000']
SE4 = var_se4['SE4_percent']
SE5 = var_se5['SE5_percent']
SEG1 = ccp_2022['SEG1_1000']
SEG2 = cvli_2022['SEG2_100000']


In [59]:
df_agregado = (
    ras_obitos_nascidos.set_index('RA', inplace=False)
    .join(S3.to_frame('S3'))
    .join(S4.to_frame('S4'))
    .join(E2.to_frame('E2'))
    .join(E5.to_frame('E5'))
    .join(E6.to_frame('E6'))
    .join(SE1.to_frame('SE1'))
    .join(SE2.to_frame('SE2'))
    .join(SE3.to_frame('SE3'))
    .join(SE4.to_frame('SE4'))
    .join(SE5.to_frame('SE5'))
    .join(SEG1.to_frame('SEG1'))
    .join(SEG2.to_frame('SEG2'))
    .join(proj_pop_ra.to_frame('POP_ESTIM'))
)

In [62]:
variaveis_df_agregado = {
    'S3': "Taxa de mortalidade fetal (por mil nascidos vivos)",
    'S4': "Taxa de mortalidade por causas evitáveis (por 10.000 habitantes)",
    'E2': "Média de alunos por turma no ensino fundamental",
    'E5': "Taxa de defasagem idade-série no ensino fundamental",
    'E6': "Taxa de reprovação no ensino fundamental",
    'SE1': "Grau de instrução dos eleitores do Brasil Esse indicador foi utilizado como proxy para o grau de escolaridade da população. Proporção da população com ensino médio ou mais concluído",
    'SE2': "Proporção de famílias em situação de pobreza (R$ < 200) que recebe benefícios sociais (Bolsa-família, BPC/LOAS, Bolsa de estudo)",
    'SE3': "Proporção de famílias em situação de extrema pobreza (R$ < 100) que recebe benefícios sociais (Bolsa-família, BPC/LOAS, Bolsa de estudo)",
    'SE4': "Porcentagem da população com 65 anos ou mais",
    'SE5': "Razão de dependência (população com menos de 15 anos e mais de 60 anos em relação à população de 15 a 60 anos)",
    'SEG1': "Taxa de ocorrências de crimes e delitos registrados pela CCP (por 1.000 habitantes)",
    'SEG2': "Taxa de CVLI (Crimes Violentos Letais Intencionais) (por 100.000 habitantes)",
    'POP_ESTIM': "População estimada na Região Administrativa (RA)"
}
df_agregado.to_csv('df_agregado.csv', sep=';', encoding='utf-8', index=True)